<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Test-runs" data-toc-modified-id="Test-runs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test runs</a></span></li><li><span><a href="#Interact" data-toc-modified-id="Interact-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Interact</a></span></li><li><span><a href="#Investigating-the-stability-of-several-models" data-toc-modified-id="Investigating-the-stability-of-several-models-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Investigating the stability of several models</a></span></li><li><span><a href="#Is-it-possible-to-have-both-constraints-on-the-transition-threshold-valid?" data-toc-modified-id="Is-it-possible-to-have-both-constraints-on-the-transition-threshold-valid?-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Is it possible to have both constraints on the transition threshold valid?</a></span></li><li><span><a href="#Stability-of-the-delta<-1-model" data-toc-modified-id="Stability-of-the-delta<-1-model-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Stability of the delta&lt; 1 model</a></span><ul class="toc-item"><li><span><a href="#delta-=-0" data-toc-modified-id="delta-=-0-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>delta = 0</a></span></li><li><span><a href="#simple-system" data-toc-modified-id="simple-system-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>simple system</a></span></li></ul></li></ul></div>

# Description

Interactive notebook to test the model and the different parameters.

# Load

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src import cell_models

import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, interact_manual

from statsmodels.tsa.stattools import acf

from scipy.signal import find_peaks

In [ ]:
from src.analysis import get_cycle_stats

In [ ]:
from src.plot_utils import run_and_plot_test

In [ ]:
from src import load_utils

# Functions

In [ ]:
# initial condition sweep to see whether the analytical solutions kind of make sense...? 

def analytical_RBc(RBc0, params, t, phase='G1'):
    
    if phase == 'G1': 
        beta = params['beta0']
    else:
        beta = params['beta0'] * params['epsilon']
    
    C1 = params['alpha']/(beta+params['gamma'])
    C2 = beta + params['gamma']
    
    return (RBc0 - C1) * np.exp(-C2*t) + C1

def analytical_M(M0, params, t):
    
    return M0 * np.exp(params['gamma']*t)

def analytical_RB(RB0, M0, params, t, phase='G1'):
    
    if phase == 'G1': 
        beta = params['beta0']
    else:
        beta = params['beta0'] * params['epsilon']
        
    C1 = params['alpha']/(beta+params['gamma'])
        
    return RB0 * np.exp(-beta*t) + C1 * M0 * (np.exp(params['gamma']*t) - np.exp(-beta*t))
    

In [ ]:
def theoretical_M_ratio(params): 

    tau = params['duration_SG2']
    A1 = np.exp(params['gamma'] * tau)/2
    EXP = - params['gamma']/(params['gamma'] + params['beta0'])
    
    C1 = params['alpha']/(params['beta0']+params['gamma'])
    C2 = params['alpha']/(params['beta0']*params['epsilon']+params['gamma'])
    
    Theta = params['transition_th']
    
    num = Theta - C1
    
    deno = (Theta + C2) * np.exp(-tau*(params['beta0'] * params['epsilon']+params['gamma'])) + C2 - C1
    
    return A1 * (num/deno)**EXP

# Test runs

Build something with interact so that you can get a rough sense of how the system behaves. 

In [ ]:
# cell_test, periods= run_and_plot_test()

# Interact

In [ ]:
interact_manual(
    run_and_plot_test, 
    alpha=fixed(.1),#(.1, 10., step), 
    beta0=(.01, .30, .01), 
    epsilon=(.1, 2., .1), #(.1, 1., step), 
    gamma=(.01, .05, .01), 
    time_SG2 = fixed(12),#(1, 15), 
    transition_th=fixed(.9), 
    delta=(0, 1, .1),
    dt=fixed(1e-1), 
    division=fixed("timer"), 
    transition=fixed("size"), # "size" or "RBc"
    k_trans=fixed(1000),
    T=fixed(10000)
);

Result for size-based transition seem to be insensitive to threshold mass, and alpha. is that true? 

# Investigating the stability of several models

In [ ]:
params=cell_models.DEFAULT_PARAMS.copy()
params['k_trans'] = 1000
params['transition'] = 'RBc' # 'size'
params['gamma'] = .01
params['beta0'] = .5
params['alpha'] = .01
params['epsilon'] = .02
params['duration_SG2'] = 66
params['delta'] = 1.


In [ ]:
limit_th = cell_models.check_conditions(params)

In [ ]:
limit_th # technically the value of transition_th that gives you M_k+1 = M_k

In [ ]:
params['transition_th'] = limit_th 

In [ ]:
cell = cell_models.cell(params=params)

cell.grow(int(5e4))

In [ ]:
_, stats = load_utils.get_phase_durations(cell)

rats = [stats['birth'][i+1]/stats['birth'][i] for i in range(len(stats['birth'])-1)]

plt.scatter(stats['birth'][:-1], rats)
plt.ylim([.8, 1.2])

plt.ylabel("Relative change in M between cycles")
plt.xlabel("Mass at birth")
plt.title(f"delta = {params['delta']}")
plt.grid()

In [ ]:
_, ax = plt.subplots()

# ic = {"RB": init_cond[0], "M": init_cond[1]}


ax.plot(cell.M_hist)
    
ax.grid()
ax.set_xlabel("t")
ax.set_ylabel("M")
# ax.set_title(f"k_trans = {params['k_trans']}")

ax.set_yscale('log')

In [ ]:
plt.plot(cell.RB_c_hist, label="sim")

t = np.linspace(0, 500*params['dt'], 500)
plt.plot(analytical_RBc(cell.RB_c_hist[0], params, t), linestyle='--', label='analytical')
plt.legend()

In [ ]:
plt.plot(cell.RB_hist)
plt.plot(analytical_RB(cell.RB_hist[0], cell.M_hist[0], params, t), linestyle='--')

In [ ]:
_, ax = plt.subplots()
ax.plot(cell.RB_c_hist)
ax.axhline([params['alpha']/(params['beta0'] + params['gamma'])], linestyle='--', color='red')
ax.axhline([params['alpha']/(params['beta0']*params['epsilon'] + params['gamma'])], linestyle='--', color='red')
ax.axhline([params['transition_th']], linestyle='--', color='green')

In [ ]:
rats_sim = []
rats_th = []
th_vec = np.linspace(0.2, 1.5, 10)


for th in th_vec: 
    params_crt = params.copy()
    params_crt['transition_th'] = th * limit_th
    
    
    rats_th.append(theoretical_M_ratio(params_crt))
    cell_ = cell_models.cell(params=params_crt)
    cell_.grow(int(5e4))
    
    _, stats = load_utils.get_phase_durations(cell_)
    rats = [stats['birth'][i+1]/stats['birth'][i] for i in range(len(stats['birth'])-1)]
    
    rats_sim.append(np.mean(rats[-10:]))
    

In [ ]:
plt.plot(rats)
plt.title("Relative growth between two cycles")

In [ ]:
_, ax = plt.subplots()
ax.plot(th_vec, rats_th, 'o--', label="theory")
ax.plot(th_vec, rats_sim, 'o--', label="simulation")
ax.axvline([1], linestyle='--', color='red')
ax.axhline([1], linestyle='--', color='red')
ax.set_xlabel("THRESHOLD")
ax.set_ylabel("relative change in M")
ax.grid()
ax.legend()
# if that is correct, the value should act as a limit value for convergence vs divergence...

# Is it possible to have both constraints on the transition threshold valid? 

In [ ]:
params=cell_models.DEFAULT_PARAMS.copy()
params['k_trans'] = 1000
params['transition'] = 'RBc' # 'size'
params['gamma'] = .01
params['beta0'] = .5
params['alpha'] = .01
params['epsilon'] = .02
params['duration_SG2'] = 10

In [ ]:
taus = np.linspace(50, 75, 100)

limit_values = []
for t in taus: 
    params_crt = params.copy()
    params_crt['duration_SG2'] = t
    
    limit_th = cell_models.check_conditions(params_crt)
    limit_values.append(limit_th)
    
    
C1 = params['alpha']/(params['beta0']+params['gamma'])
C2 = params['alpha']/(params['epsilon']*params['beta0']+params['gamma'])

_, ax = plt.subplots()
ax.plot(taus, limit_values)
ax.axhline([C1], color='red', linestyle='--')
ax.axhline([C2], color='red', linestyle='--')

ax.set_xlabel("t_sg2")
ax.set_ylim([.9*C1, 1.5*C2])

We can take a duration of 60 hours

# Stability of the delta< 1 model

We observe that the models with delta<1 are stable oscillators. Why? 

## delta = 0

In this case with the implicit function theorem we can map the derivative of the ratio of masses at G1 as a function of the mass at G1. This derivative looks to be always negative... is that even true? 

In [ ]:
def dgdy(z, y, alpha, beta, gamma, epsilon, Omega, tau2): 
    A = -gamma * tau2 / y**2
    B = -gamma/beta * (Omega * z / (Omega * z * y - alpha/beta) - (Omega * np.exp(-epsilon*beta*tau2)))/((Omega * y - alpha/epsilon/beta) * np.exp(-epsilon*beta*tau2) + alpha/epsilon/beta - alpha/beta)
    return A + B

In [ ]:
def dgdz(z, y, alpha, beta, gamma, epsilon, Omega, tau2):
    return -1-gamma/(beta*z-alpha/Omega/y)

In [ ]:
alpha = 1
beta = .2
gamma = .4
epsilon = .2
Omega = .1
tau2 = 12

In [ ]:
n_points = 1000

In [ ]:
z_vec = np.linspace(1e-5, 1, n_points)

y_vec = np.linspace(1e-5, 1000, n_points)

In [ ]:
dgdy_mat = np.zeros((n_points, n_points))

dgdz_mat = np.zeros((n_points, n_points))

In [ ]:
for (k, z) in enumerate(z_vec):
    dgdy_mat[k, :] = dgdy(z, y_vec, alpha, beta, gamma, epsilon, Omega, tau2)
    dgdz_mat[k, :] = dgdz(z, y_vec, alpha, beta, gamma, epsilon, Omega, tau2)

In [ ]:
dzdy = dgdz_mat**(-1) * dgdy_mat

In [ ]:
dzdy_sign = dzdy>0

In [ ]:
plt.imshow(dzdy_sign)

There is a surface with instability. So probably if you initialize your system there, you can reach this surface, right? You probably want to plot the flow, right? 

Actually, how can we think of the stability of a system that acts by multiplication? 

## simple system

In [ ]:
y = lambda a, b, x: a*x+b

In [ ]:
y2 = lambda a, b, x: np.exp(-a*x)+b

In [ ]:
n_steps = 10000

a = 3
b = .4

sk = .1

sk_list = []
yk_list=[]

# run iterations
for k in range(n_steps): 
    yk = y2(a, b, sk)
    sk = yk*sk
    
    sk_list.append(sk)
    yk_list.append(yk)

In [ ]:
plt.plot(sk_list)

In [ ]:
plt.plot(yk_list)

In [ ]:
plt.plot(xx, y2(a, b, xx))
plt.scatter(sk_list, yk_list)